# Concise Implementation of Linear Regression

In [1]:
!pip install -U mxnet-cu101==1.7.0

     |███████████████████████████████▌| 834.1 MB 1.4 MB/s eta 0:00:09tcmalloc: large alloc 1147494400 bytes == 0x5597f4d24000 @  0x7efe53694615 0x5597baf954cc 0x5597bb07547a 0x5597baf982ed 0x5597bb089e1d 0x5597bb00be99 0x5597bb0069ee 0x5597baf99bda 0x5597bb00bd00 0x5597bb0069ee 0x5597baf99bda 0x5597bb008737 0x5597bb08ac66 0x5597bb007daf 0x5597bb08ac66 0x5597bb007daf 0x5597bb08ac66 0x5597bb007daf 0x5597baf9a039 0x5597bafdd409 0x5597baf98c52 0x5597bb00bc25 0x5597bb0069ee 0x5597baf99bda 0x5597bb008737 0x5597bb0069ee 0x5597baf99bda 0x5597bb007915 0x5597baf99afa 0x5597bb007c0d 0x5597bb0069ee
     |████████████████████████████████| 846.0 MB 19 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
!pip install d2l==0.17.0

     |████████████████████████████████| 83 kB 1.1 MB/s 


In [3]:
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# Reading Dataset

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #
    """Construct a Gluon data iterator."""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[array([[ 0.1737154 , -1.3096327 ],
        [-0.71561056, -1.0295794 ],
        [ 0.93694544, -0.20085481],
        [ 0.2552471 ,  0.19742578],
        [-1.88909   , -2.515245  ],
        [ 1.3225973 ,  0.57039356],
        [-0.01125004,  0.999566  ],
        [ 0.24499686, -1.6080191 ],
        [ 1.015367  ,  1.2454321 ],
        [ 0.91511744,  0.84187603]]), array([[ 9.009764 ],
        [ 6.2538333],
        [ 6.7493343],
        [ 4.058058 ],
        [ 8.953278 ],
        [ 4.914644 ],
        [ 0.7754266],
        [10.171097 ],
        [ 2.0036945],
        [ 3.1702974]])]

# Defining the Model

In [6]:
# `nn` is an abbreviation for neural networks
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(1))

# Initializing Model Parameters

In [7]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

# Defining the Loss Function

In [8]:
loss = gluon.loss.L2Loss()

# Defining the Optimization Algorithm

In [9]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

# Training

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')

epoch 1, loss 0.025005
epoch 2, loss 0.000092
epoch 3, loss 0.000051


In [11]:
w = net[0].weight.data()
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(f'error in estimating b: {true_b - b}')

error in estimating w: [ 0.00030911 -0.0002439 ]
error in estimating b: [0.00025797]
